In [1]:
import numpy as np
import torch
import pandas as pd
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [2]:
data = pd.read_csv('C:/Users/luizf/Desktop/MINDS/Datasets/air_quality_beijing_6_sites.csv', sep=',')
data = data.drop(labels=['No','day','year','month','hour','wd','station'], axis=1)
data = data.drop(labels=['No.1','day.1','year.1','month.1','hour.1','wd.1','station.1'], axis=1)
data = data.drop(labels=['No.2','day.2','year.2','month.2','hour.2','wd.2','station.2'], axis=1)
data = data.drop(labels=['No.3','day.3','year.3','month.3','hour.3','wd.3','station.3'], axis=1)
data = data.drop(labels=['No.4','day.4','year.4','month.4','hour.4','wd.4','station.4'], axis=1)
data = data.drop(labels=['No.5','day.5','year.5','month.5','hour.5','wd.5','station.5'], axis=1)
data.dropna(inplace=True)
#data.loc[:,['CO(GT)', 'C6H6(GT)', 'T', 'RH', 'AH']] = data.loc[:,['CO(GT)', 'C6H6(GT)', 'T', 'RH', 'AH']].applymap(lambda x: str(x.replace(',','.')))
data = data.astype(float)
X=data.copy()
X.drop(data.tail(1).index,inplace=True)
X
y=data.copy()
y.loc[max(y.index)+1, :] = None
y=y.shift(-1)
y.dropna(inplace=True)
y

,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,...,PM10.5,SO2.5,NO2.5,CO.5,O3.5,TEMP.5,PRES.5,DEWP.5,RAIN.5,WSPM.5
23,22.0,24.0,24.0,44.0,500.0,44.0,-0.4,1031.0,-17.6,0.0,...,18.0,24.0,43.0,1399.0,25.0,-0.3,1028.9,-18.0,0.0,1.4
24,14.0,17.0,21.0,36.0,400.0,50.0,-1.0,1031.3,-17.3,0.0,...,39.0,25.0,28.0,1300.0,37.0,-0.7,1029.2,-17.9,0.0,1.0
25,9.0,11.0,19.0,70.0,500.0,20.0,-1.8,1030.1,-17.5,0.0,...,23.0,26.0,11.0,1200.0,50.0,-1.3,1028.0,-18.4,0.0,1.4
29,3.0,11.0,34.0,58.0,500.0,27.0,-1.7,1029.8,-17.0,0.0,...,6.0,19.0,11.0,1100.0,52.0,-1.3,1028.0,-17.9,0.0,0.6
31,3.0,7.0,21.0,49.0,500.0,43.0,-0.4,1029.6,-17.6,0.0,...,9.0,22.0,15.0,1200.0,51.0,-0.6,1027.7,-17.8,0.0,0.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35054,12.0,23.0,6.0,13.0,200.0,120.0,14.2,1012.5,-14.9,0.0,...,47.0,3.0,4.0,400.0,69.0,14.1,1009.0,-17.1,0.0,4.1
35057,13.0,37.0,7.0,45.0,500.0,81.0,11.6,1013.6,-15.1,0.0,...,83.0,6.0,60.0,700.0,26.0,11.1,1010.4,-15.5,0.0,1.9
35060,16.0,37.0,10.0,66.0,700.0,58.0,10.8,1014.2,-13.3,0.0,...,37.0,7.0,52.0,600.0,27.0,10.5,1010.8,-15.9,0.0,2.3
35061,21.0,44.0,12.0,87.0,700.0,35.0,10.5,1014.4,-12.9,0.0,...,23.0,3.0,13.0,400.0,57.0,8.9,1010.9,-14.9,0.0,1.6


In [3]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train, X_test = train_test_split(X, test_size=0.25, shuffle=False)
X_train, X_val = train_test_split(X_train, test_size=0.15, shuffle=False)
X_train = scaler.fit_transform(X_train.values)
X_val = scaler.fit_transform(X_val.values)
train_loader = torch.utils.data.DataLoader(torch.tensor(X_train.astype(np.float32)), batch_size=32)
val_loader = torch.utils.data.DataLoader(torch.tensor(X_val.astype(np.float32)), batch_size=32)

y_train, y_test = train_test_split(y, test_size=0.25, shuffle=False)
y_train, y_val = train_test_split(y_train, test_size=0.15, shuffle=False)
y_train = scaler.fit_transform(y_train.values)
y_val = scaler.fit_transform(y_val.values)

In [4]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout_prob):
        super(LSTMModel, self).__init__()

        # Defining the number of layers and the nodes in each layer
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim

        # LSTM layers
        self.lstm = nn.LSTM(
            input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob
        )

        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initializing hidden state for first input with zeros
        if torch.cuda.is_available():
            h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_().cuda()
        else:
            h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # Initializing cell state for first input with zeros
        if torch.cuda.is_available():
            c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_().cuda()
        else:
            c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()
        

        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        # Forward propagation by passing in the input, hidden state, and cell state into the model
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        # Reshaping the outputs in the shape of (batch_size, seq_length, hidden_size)
        # so that it can fit into the fully connected layer
        out = out[:, -1, :]

        # Convert the final state to our desired output shape (batch_size, output_dim)
        out = self.fc(out)

        return out

In [5]:
def train_lstm(X_train, layer_dim, dropout_prob, epochs, batch_size, learning_rate, step_wdecay, gamma_wdecay, X_val=0):
    train_loader = torch.utils.data.DataLoader(torch.tensor(X_train.astype(np.float32)), batch_size=batch_size, shuffle=False)
    #val_loader = torch.utils.data.DataLoader(torch.tensor(X_val.astype(np.float32)), batch_size=batch_size,shuffle=False)
    if torch.cuda.is_available():
        y_train_t= torch.tensor(y_train.astype(np.float32)).cuda()
        #y_val_t = torch.tensor(y_val.astype(np.float32)).cuda()
    else:
        y_train_t = torch.tensor(y_train.astype(np.float32))
        #y_val_t = torch.tensor(y_val.astype(np.float32))
    torch.manual_seed(1)
    alpha = 0.5
    h=round(X_train.shape[0]/(alpha * (X_train.shape[1]*2)))
    model = LSTMModel(X_train.shape[1],h, layer_dim,X_train.shape[1],dropout_prob)
    if torch.cuda.is_available():
        model = model.cuda()
    else:
        model = model.cuda()
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    buffer = 10000
    steplr = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_wdecay, gamma=gamma_wdecay)

    min_valid_loss=500
    for epoch in range(epochs):
        aux,aux2=0,0
        train_loss=0.0
        model.train()
        for batch, d in enumerate(train_loader):
            if torch.cuda.is_available():
                d =d.cuda()
            optimizer.zero_grad()  #set gradient to 0
            output = model(d.reshape(-1, d.shape[1]).unsqueeze(1))  #make a prediction
            if torch.cuda.is_available():
                loss = criterion(output, y_train_t[aux:aux+d.shape[0]])
            else:
                loss = criterion(output, torch.tensor(y_train_t[aux:aux+d.shape[0]].astype(np.float32)))  #calculate the loss
            loss.backward()  #differentiate loss with respect to parameters
            optimizer.step() #apply the method step and updates the parameters 
            train_loss += loss.item()
            aux+=d.shape[0]
        steplr.step()

        #valid_loss = 0.0
        #model.eval() 
        #for dv in val_loader:  
        #    if torch.cuda.is_available():
        #        dv =dv.cuda()  
        #    target = model(dv.unsqueeze(1))
        #    if torch.cuda.is_available():
        #        val_loss = criterion(target, y_val_t[aux2:aux2+dv.shape[0]])
        #    else:
        #        val_loss = criterion(target,torch.tensor(y_val_t[aux2:aux2+dv.shape[0]].astype(np.float32)))
        #    valid_loss += val_loss.item()
        #    aux2+=dv.shape[0]
        #print(f'Epoch {epoch+1} \t\t Training Loss: {train_loss / len(train_loader)} \t\t Validation Loss: {valid_loss / len(val_loader)}')
        #if valid_loss < buffer:
        #    buffer = valid_loss
#
   # print (buffer/len(val_loader))   
    return model

In [6]:
def cal_nrmse(rmse, y):
    x = max(y)-min(y)
    return (rmse/x)

In [7]:
n_windows = 30
w = round(len(X)/n_windows)
buffer =  {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape":[],
         "nrmse":[],
         "variable":[]
    }
    
col = ["window","rmse","mape","mae","r2","variable"]
result = pd.DataFrame(columns=col)
a={}
for i in range(n_windows):
    dfx=X.iloc[int(w*i):int(w*(i+1)), :]
    dfy=y.iloc[int(w*i):int(w*(i+1)), :]
    scaler = StandardScaler()
    X_train, X_test = train_test_split(dfx, test_size=0.25, shuffle=False)
    #X_train, X_val = train_test_split(X_train, test_size=0.15, shuffle=False)
    X_train = scaler.fit_transform(X_train.values)
    #X_val = scaler.fit_transform(X_val.values)


    y_train, y_test = train_test_split(dfy, test_size=0.25, shuffle=False)
    #y_train, y_val = train_test_split(y_train, test_size=0.15, shuffle=False)
    y_train = scaler.fit_transform(y_train.values)
    #y_val = scaler.fit_transform(y_val.values)

    model = train_lstm(X_train, layer_dim=1, dropout_prob=0, epochs=300, batch_size=64, learning_rate=0.001, step_wdecay=50, gamma_wdecay=0.5)

    model.eval()
    if torch.cuda.is_available():
        test_norm= torch.tensor(scaler.fit_transform(X_test.values.astype(np.float32))).cuda()
    else:
        test_norm= torch.tensor(scaler.fit_transform(X_test.values.astype(np.float32)))
    y_norm = scaler.fit_transform(y_test.values)
    #yhat=scaler.inverse_transform(model(test_norm.unsqueeze(1)).detach().numpy())
    yhat=scaler.inverse_transform(model(test_norm.unsqueeze(1)).detach().cpu().numpy())
    #y=scaler.inverse_transform(test_norm.detach().numpy())[:,0]
    
    from pyFTS.benchmarks import Measures
    buffer['rmse']=[Measures.rmse(y_test.values[:,j],yhat[:,j]) for j in range (X.shape[1])]
    buffer['mape']=[Measures.mape(y_test.values[:,j],yhat[:,j]) for j in range (X.shape[1])]
    buffer['mae']=[mean_absolute_error(y_test.values[:,j],yhat[:,j]) for j in range (X.shape[1])]
    buffer['r2']=[r2_score(y_test.values[:,j],yhat[:,j]) for j in range (X.shape[1])]
    buffer['smape']=[Measures.smape(y_test.values[:,j],yhat[:,j]) for j in range (X.shape[1])]
    buffer['nrmse']=[cal_nrmse(Measures.rmse(y_test.values[:,j],yhat[:,j]),y_test.values[:,j]) for j in range (X.shape[1])]
    buffer['variable']=[j for j in X.columns]
    buffer['window']= w*i

    result = pd.concat([result, pd.DataFrame(buffer)], ignore_index=True)
    #result['rmse'].append([j for j in buffer['rmse']])
    #result['mape'].append(buffer['mape'])
    #result['mae'].append(buffer['mae'])
    #result['r2'].append(buffer['r2'])
    #result['variable'].append([j for j in X.columns])
    #result['window'].append(buffer['window'])
#
#result = np.mean(buffer)
#print (result)


    
    


C:\Users\luizf\miniconda3\envs\test\lib\site-packages\pyFTS\benchmarks\Measures.py:71: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
C:\Users\luizf\AppData\Local\Temp/ipykernel_13872/1568283450.py:3: RuntimeWarning: divide by zero encountered in double_scalars
  return (rmse/x)
C:\Users\luizf\miniconda3\envs\test\lib\site-packages\pyFTS\benchmarks\Measures.py:71: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
C:\Users\luizf\miniconda3\envs\test\lib\site-packages\pyFTS\benchmarks\Measures.py:71: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
C:\Users\luizf\AppData\Local\Temp/ipykernel_13872/1568283450.py:3: RuntimeWarning: divide by zero encountered in double_scalars
  return (rmse/x)
C:\Users\lui

In [8]:
result

,window,rmse,mape,mae,r2,variable,smape,nrmse
0,0,34.197148,72.092924,22.742087,0.582019,PM2.5,35.066363,0.133583
1,0,44.825185,48.254920,33.145090,0.549091,PM10,18.226023,0.148921
2,0,11.269470,56.279570,7.428109,0.450483,SO2,22.976291,0.112695
3,0,20.856401,35.914753,14.720816,0.418191,NO2,15.730526,0.175264
4,0,232.028031,20.154646,152.884187,0.480539,CO,9.850515,0.154685
...,...,...,...,...,...,...,...,...
1975,22475,4.826588,281.071161,4.194963,0.160915,TEMP.5,54.754438,0.243767
1976,22475,3.609606,0.267764,2.715462,0.674181,PRES.5,0.133921,0.127548
1977,22475,3.438943,24.806623,2.765514,0.529296,DEWP.5,11.424829,0.169406
1978,22475,0.067647,inf,0.019081,0.009107,RAIN.5,99.761200,0.112744


In [9]:
final_measures = {'variable':[], 'rmse': [], 'mae': [], 'mape': [], 'r2': [], 'smape':[], 'nrmse':[], 'rmse_std': [], 'mae_std': [], 'mape_std': [], 'r2_std': [], 'smape_std':[], 'nrmse_std':[]}
m=[]
for i in list(X.columns):
    m=result[result.variable==i].mean()
    st = result[result.variable==i].std()
    final_measures['variable'].append(i)
    final_measures['rmse'].append(m['rmse'])
    final_measures['mae'].append(m['mae'])
    final_measures['mape'].append(m['mape'])
    final_measures['r2'].append(m['r2'])
    final_measures['smape'].append(m['smape'])
    final_measures['nrmse'].append(m['nrmse'])
    final_measures['rmse_std'].append(st['rmse'])
    final_measures['mae_std'].append(st['mae'])
    final_measures['mape_std'].append(st['mape'])
    final_measures['r2_std'].append(st['r2'])
    final_measures['smape_std'].append(st['smape'])
    final_measures['nrmse_std'].append(st['nrmse'])
pd.DataFrame(final_measures)

C:\Users\luizf\AppData\Local\Temp/ipykernel_13872/2831257117.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  m=result[result.variable==i].mean()
C:\Users\luizf\AppData\Local\Temp/ipykernel_13872/2831257117.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  st = result[result.variable==i].std()


,variable,rmse,mae,mape,r2,smape,nrmse,rmse_std,mae_std,mape_std,r2_std,smape_std,nrmse_std
0,PM2.5,35.935706,25.962886,72.311730,0.697405,23.969895,0.129029,15.998481,10.675671,40.488885,0.132851,7.321701,0.018422
1,PM10,54.050134,38.347398,71.072362,0.562980,22.988742,0.138429,30.134858,18.514527,25.787954,0.249755,6.690002,0.020629
2,SO2,9.030672,6.538360,77.916512,0.363796,25.068672,0.158157,6.767603,4.802477,36.581334,0.160142,7.310142,0.038342
3,NO2,21.208911,16.511891,42.855836,0.527625,15.749098,0.150072,4.721856,3.871118,15.624346,0.150603,2.572825,0.023398
4,CO,512.669314,377.448614,43.438724,0.560049,16.959346,0.133976,273.833291,212.425686,17.726133,0.155716,4.416780,0.019481
...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,TEMP.5,341.344770,259.050313,inf,0.554170,23.732920,0.149532,1490.329490,1112.079825,NaN,0.219262,24.316129,0.039722
62,PRES.5,3.005094,2.369277,0.234508,0.547935,0.117240,0.157712,1.516438,1.229136,0.120913,0.189337,0.060436,0.027567
63,DEWP.5,3.365571,2.666439,inf,0.556153,22.279781,0.164717,1.700066,1.393412,NaN,0.159763,17.686357,0.035475
64,RAIN.5,0.431115,0.165604,inf,0.028580,98.074681,inf,0.779851,0.259969,NaN,0.087129,3.400624,NaN


In [10]:
pd.DataFrame(final_measures).to_csv('lstm_beijing6_result.csv')